In [13]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn

In [18]:
from timm.models.vision_transformer import PatchEmbed, Block

In [14]:

def get_2d_sincos_pos_embed(embed_dim, grid_size, cls_token=False):
    """
    grid_size: int of the grid height and width
    return:
    pos_embed: [grid_size*grid_size, embed_dim] or [1+grid_size*grid_size, embed_dim] (w/ or w/o cls_token)
    """
    grid_h = np.arange(grid_size, dtype=np.float32)
    grid_w = np.arange(grid_size, dtype=np.float32)
    grid = np.meshgrid(grid_w, grid_h)  # here w goes first
    grid = np.stack(grid, axis=0)

    grid = grid.reshape([2, 1, grid_size, grid_size])
    pos_embed = get_2d_sincos_pos_embed_from_grid(embed_dim, grid)
    if cls_token:
        pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis=0)
    return pos_embed

def get_2d_sincos_pos_embed_from_grid(embed_dim, grid):
    assert embed_dim % 2 == 0

    # use half of dimensions to encode grid_h
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0])  # (H*W, D/2)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1])  # (H*W, D/2)

    emb = np.concatenate([emb_h, emb_w], axis=1) # (H*W, D)
    return emb



def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=np.float)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2,)

    pos = pos.reshape(-1)  # (M,)
    out = np.einsum('m,d->md', pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out) # (M, D/2)
    emb_cos = np.cos(out) # (M, D/2)

    emb = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)
    return emb


def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=np.float)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2,)

    pos = pos.reshape(-1)  # (M,)
    out = np.einsum('m,d->md', pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out) # (M, D/2)
    emb_cos = np.cos(out) # (M, D/2)

    emb = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)
    return emb

In [28]:
class MaskedAutoencoderViT(nn.Module):
    def __init__(self, img_size = 32, patch_size = 16, in_chans = 3, embed_dim = 1024, depth = 24, num_heads = 16, decoder_embed_dim = 512, decoder_depth = 8,\
        decoder_num_heads = 16, mlp_ratio = 4, norm_layer = nn.LayerNorm, norm_pix_loss = False):
        super().__init__()
        self.patch_embed = PatchEmbed(img_size = 32, patch_size=4, in_chans=3, embed_dim=1024)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim), requires_grad=False)

        self.blocks = nn.ModuleList([
            Block(embed_dim, num_heads, mlp_ratio, qkv_bias = True, qk_scale = None, norm_layer = norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        #decoder
        self.decoder_embed = nn.Linear(embed_dim, decoder_embed_dim, bias = True)

        self.mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))

        self.decoder_pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, decoder_embed_dim), requires_grad=False)

        self.decoder_blocks = nn.ModuleList([
            Block(decoder_embed_dim, decoder_num_heads, mlp_ratio, qkv_bias = True, qk_scale = None, norm_layer = norm_layer)
            for i in range(decoder_depth)])
        
        self.decoder_norm = norm_layer(decoder_embed_dim)

        self.decoder_pred = nn.Linear(decoder_embed_dim, patch_size**2, in_chans, bias = True)

        self.norm_pix_loss = norm_pix_loss

        self.initialize_weights()
    
    def initializa_weights(self):
        pos_embed = get_2d_sincos_pos_embed(self.pos_embed.shape[-1], self.patch_embed.shape[-1], int(np.sqrt(self.patch_embed.shape[-1])), cls_token = True)
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueese(0))
        
        decoder_pos_embed = get_2d_sincos_pos_embed(self.decoder_pos_embed.shape[-1], self.patch_embed.shape[-1], int(np.sqrt(self.patch_embed.shape[-1])), cls_token = True)
        self.decoder_pos_embed.data._(torch.from_numpy(decoder_pos_embed).float().unsqueese(0))

        w = self.patch_embed.proj.weight.data
        torch.nn.init.xavier_uniform_(w.view([w.shape[0], -1]))

        torch.nn.init.normal_(self.cls_token, std=0.02)
        torch.nn.init.normal_(self.mask_token, std = 0.02)

        self.apply(self._init_weights)

        
    def patchify(self, imgs):
        """
        imgs: (N, 3, H, W)
        x: (N, L, patch_size**2 *3)
        """
        p = self.patch_embed.patch_size[0]
        assert imgs.shape[2] == imgs.shape[3] and imgs.shape[2] % p == 0

        h = w = imgs.shape[2] // p
        x = imgs.reshape(shape=(imgs.shape[0], 3, h, p, w, p))
        x = torch.einsum('nchpwq->nhwpqc', x)
        x = x.reshape(shape=(imgs.shape[0], h * w, p**2 * 3))
        return x

    def unpatchify(self, x):
        """
        x: (N, L, patch_size**2 *3)
        imgs: (N, 3, H, W)
        """
        p = self.patch_embed.patch_size[0]
        h = w = int(x.shape[1]**.5)
        assert h * w == x.shape[1]
        
        x = x.reshape(shape=(x.shape[0], h, w, p, p, 3))
        x = torch.einsum('nhwpqc->nchpwq', x)
        imgs = x.reshape(shape=(x.shape[0], 3, h * p, h * p))
        return imgs
    
    



In [35]:
 noise = torch.rand(10, 5)  # noise in [0, 1]

# sort noise for each sample
ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
ids_restore = torch.argsort(ids_shuffle, dim=1)


In [39]:
ids_shuffle

tensor([[0, 3, 1, 4, 2],
        [3, 2, 0, 4, 1],
        [4, 1, 2, 3, 0],
        [4, 3, 1, 0, 2],
        [0, 3, 4, 2, 1],
        [0, 4, 2, 1, 3],
        [0, 1, 4, 2, 3],
        [2, 3, 0, 1, 4],
        [3, 0, 1, 2, 4],
        [3, 1, 4, 0, 2]])

In [40]:
mask = torch.ones([10, 5])
mask[:, :3] = 0
# unshuffle to get the binary mask
mask = torch.gather(mask, dim=1, index=ids_restore)

In [41]:
mask

tensor([[0., 0., 1., 0., 1.],
        [0., 1., 0., 0., 1.],
        [1., 0., 0., 1., 0.],
        [1., 0., 1., 0., 0.],
        [0., 1., 1., 0., 0.],
        [0., 1., 0., 1., 0.],
        [0., 0., 1., 1., 0.],
        [0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1.],
        [1., 0., 1., 0., 0.]])

In [16]:
torch.randn(64, 1024).unsqueeze(0).shape

torch.Size([1, 64, 1024])

In [24]:
w = PatchEmbed(32, 4, 3, 512).proj.weight.data

In [26]:
w.shape

torch.Size([512, 3, 4, 4])

In [ ]:
torch.nn.init.xavier_normal_()

In [27]:
w.view([w.shape[0], -1]).shape

torch.Size([512, 48])

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
n_gpu = torch.cuda.device_count()

In [4]:
n_gpu

0

In [8]:
torch.cuda.is_available()

False

In [6]:
torch.zeros(1).cuda()

AssertionError: Torch not compiled with CUDA enabled